## Observer Design

Let consider the DC motor model

In [25]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

J = 0.01;
b = 0.1;
K = 0.01;
R = 1;
L = 0.5;

A = np.array([[-b/J, K/J],[-K/L, -R/L]])
B = np.array([[0],[1/L]])
C = np.array([[1, 0]])

A = np.matrix(A)
B = np.matrix(B)
C = np.matrix(C)

n=A.shape[0]


If you want to know about the physical meaning of the parameters, please see the website
"https://ctms.engin.umich.edu/CTMS/index.php?example=MotorSpeed&section=SystemModeling"

Based on the above DC motor model, one can design a Luenburger observer as follows:
\begin{align}
&\dot{\hat{x}}(t)= A\hat{x}(t) +Bu(t) + L (y(t)-\hat{y}(t)),\tag{1}\\
&y=Cx(t), \hat{y}=C\hat{x}(t),
\end{align}
where $L$ denotes the Luenberger observer gain.

The error system between a plant (the DC motor) and the oberver (1) is represented as 
\begin{align}
\dot{e}(t) = (A-LC)e(t),
\end{align}
where $e(t)=x(t)-\hat{x}(t)$.

Now, we will focus on designing the $L$ gain. The LMI condition for designing the $L$ can be considered as the following code.

In [33]:

Lbar = cp.Variable((n,B.shape[1]))
P = cp.Variable((n,n), PSD=True)
I = np.identity(n)

objective = cp.Minimize(0)
prob = cp.Problem(objective, [A.T*P.T-C.T*Lbar.T+P*A-Lbar*C + 0.001*X <= 0, X - 0.11*I >> 0])
prob.solve()

0.0

Let check the stability of the error system and obtain the $L$ gain.

In [36]:
Pop = P.value
Lop = Lbar.value

Pinv = np.linalg.inv(Pop)
L = np.matmul(Pinv,Lop)

[sys_stab,sys_vec] = np.linalg.eig(A-L@C)
print(L)
print(sys_stab)

[[143.36323944]
 [ 36.28416772]]
[-153.12301018   -2.24022925]


### Simulation

In [ ]:
def plant(x,u):    
    J = 0.01;
    b = 0.1;
    K = 0.01;
    R = 1;

    dx[0] = -(b/J)*x[0] + (K/J)*x[1] 
    dx[1] = -(K/L)*x[0] - (R/L)*x[1] + (1/L)*u
    return dx

def rk5(x,u,T):
    k1=plant(x,u)*T
    k2=plant(x+k1*0.5,u)*T
    k3=plant(x+k2*0.5,u)*T
    k4=plant(x+k3,u)*T
    dx = x + ((k1+k4)/6+(k2+k3)/3)
    return dx

x0 = np.array([[0.1],[-0.1]])
dx = np.zeros([3,1])
temp_s = np.zeros([3,1])
T=0.001
tf=1
sam=int(tf/T)
tspan = np.linspace(0,tf, sam+1)

xs=len(tspan)
x=np.zeros([n,xs])
u_sig=np.zeros([B.shape[1],xs])
x[0,0]=x0[0]
x[1,0]=x0[1]
x[2,0]=x0[2]

u_sig[:,0] = np.matmul(K,x[:,0])

for i in range(0,xs-1):
    u = np.matmul(K,x[:,i])

    temp_s[0] = x[0,i]
    temp_s[1] = x[1,i]
    temp_s[2] = x[2,i]
    
    x_next = rk5(temp_s,u,T)
    x[0,i+1] = x_next[0]
    x[1,i+1] = x_next[1]
    x[2,i+1] = x_next[2]
  
    u_sig[:,i+1] = u
plt.figure()
plt.plot(tspan, x[0,:], label = "x1")
plt.plot(tspan, x[1,:], label = "x2")
plt.plot(tspan, x[2,:], label = "x3")
plt.grid()
plt.xlabel("Time")
plt.ylabel("State response")
plt.legend()
plt.show()

plt.figure()
plt.plot(tspan, u_sig[0,:], label = "u")
plt.grid()
plt.xlabel("Time")
plt.ylabel("Control signal")
plt.legend()
plt.show()